## Preprocessing

In [10]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.layers import Activation, Dense

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [50]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

# Increasing cutoff to from 100 to 1000 made the accuracy go up from 72.7 to73.35

In [11]:
application_df.describe()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


In [12]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [13]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [14]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T25','T14','T29','T15','T17','T2','T12','T13','T9','T10','T7','T8']
cutoff = 1000

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [16]:
# Look at CLASSIFICATION value counts for binning
classif = application_df['CLASSIFICATION'].value_counts()
classif

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [17]:
low_classif = list(classif.loc[classif < 1000].index)

In [18]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace =  low_classif
classification_cut = 1000

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df)

In [20]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=8, activation = "relu", input_dim= len(X_train[0])))

# Second hidden layer
nn.add(Dense(units=4, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 328       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 369 (1.44 KB)
Trainable params: 369 (1.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [24]:
# Train the model
nn_fitted = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 783us/step - loss: 0.6430 - accuracy: 0.6617
Epoch 2/100
804/804 [==============================] - 1s 772us/step - loss: 0.5895 - accuracy: 0.7177
Epoch 3/100
804/804 [==============================] - 1s 793us/step - loss: 0.5710 - accuracy: 0.7250
Epoch 4/100
804/804 [==============================] - 1s 779us/step - loss: 0.5660 - accuracy: 0.7264
Epoch 5/100
804/804 [==============================] - 1s 780us/step - loss: 0.5639 - accuracy: 0.7266
Epoch 6/100
804/804 [==============================] - 1s 800us/step - loss: 0.5624 - accuracy: 0.7281
Epoch 7/100
804/804 [==============================] - 1s 775us/step - loss: 0.5613 - accuracy: 0.7285
Epoch 8/100
804/804 [==============================] - 1s 837us/step - loss: 0.5607 - accuracy: 0.7279
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7277
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss:

804/804 [==============================] - 1s 784us/step - loss: 0.5532 - accuracy: 0.7316
Epoch 80/100
804/804 [==============================] - 1s 801us/step - loss: 0.5530 - accuracy: 0.7319
Epoch 81/100
804/804 [==============================] - 1s 790us/step - loss: 0.5528 - accuracy: 0.7316
Epoch 82/100
804/804 [==============================] - 1s 791us/step - loss: 0.5526 - accuracy: 0.7313
Epoch 83/100
804/804 [==============================] - 1s 791us/step - loss: 0.5523 - accuracy: 0.7321
Epoch 84/100
804/804 [==============================] - 1s 806us/step - loss: 0.5528 - accuracy: 0.7320
Epoch 85/100
804/804 [==============================] - 1s 790us/step - loss: 0.5528 - accuracy: 0.7323
Epoch 86/100
804/804 [==============================] - 1s 790us/step - loss: 0.5530 - accuracy: 0.7314
Epoch 87/100
804/804 [==============================] - 1s 793us/step - loss: 0.5527 - accuracy: 0.7322
Epoch 88/100
804/804 [==============================] - 1s 802us/step - loss:

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5540 - accuracy: 0.7335 - 250ms/epoch - 933us/step
Loss: 0.5540236234664917, Accuracy: 0.7335277199745178


# changing number of nodes from 8 and 4 to 512 and 256 increased accuracy from 73.35 to 73.48

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=512, activation = "relu", input_dim= len(X_train[0])))

# Second hidden layer
nn.add(Dense(units=256, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               20992     
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 152577 (596.00 KB)
Trainable params: 152577 (596.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [29]:
nn_fitted = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5748 - accuracy: 0.7179
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.7262
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5587 - accuracy: 0.7281
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7289
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7291
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7314
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7310
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7371
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5408 - accuracy: 0.7364
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7375
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7365
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7376
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7375
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5402 - accuracy: 0.7371
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7374
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5425 - accuracy: 0.7376
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 

In [30]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5705 - accuracy: 0.7348 - 270ms/epoch - 1ms/step
Loss: 0.5704584717750549, Accuracy: 0.734810471534729


# adding another hidden layer increased the accuracy from 73.48 to 73.63

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=512, activation = "tanh", input_dim= len(X_train[0])))

# Second hidden layer
nn.add(Dense(units=256, activation = "relu"))

nn.add(Dense(units= 128, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 512)               20992     
                                                                 
 dense_18 (Dense)            (None, 256)               131328    
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 185345 (724.00 KB)
Trainable params: 185345 (724.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [46]:
nn_fitted = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5760 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5644 - accuracy: 0.7259
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5617 - accuracy: 0.7264
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5590 - accuracy: 0.7280
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5589 - accuracy: 0.7284
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7289
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7295
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7303
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7299
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7371
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7374
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7368
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7375
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7382
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7376
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5407 - accuracy: 0.7373
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7377
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5399 - accuracy: 0.7379
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 

In [47]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5740 - accuracy: 0.7363 - 289ms/epoch - 1ms/step
Loss: 0.5740053653717041, Accuracy: 0.7363265156745911


In [48]:
nn.save("AlphabetSoupCharity_Optimization.h5")